In [1]:
import Pkg
Pkg.activate("..")

  Activating project at `~/Projects/latentplan.jl`


In [2]:
using Test
using PyCall
using Knet
using Debugger: @enter, @bp, @run
using CUDA

if CUDA.functional()
	atype=KnetArray{Float32}
else	
	atype=Array{Float32}
end
cputype=Array{Float32}

Array{Float32}

In [3]:
check(v) = any(isnan.(cputype(v))) || any(isinf.(cputype(v)))

check (generic function with 1 method)

In [4]:
include("datasets/sequence.jl")
include("models/common.jl")
include("models/transformers.jl")
include("models/vqvae.jl")
include("setup.jl")

No module named 'flow'
/home/mehmeteneserciyes/.julia/conda/3/lib/python3.10/site-packages/glfw/__init__.py:912: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
No module named 'carla'
pybullet build time: Dec 17 2022 18:25:17


read_config (generic function with 1 method)

In [5]:
losssum(prediction) = mean(prediction[2] + prediction[3] + prediction[4])

losssum (generic function with 1 method)

In [6]:
function zerograd_embedding(model::VQContinuousVAE)
    model.model.codebook.embedding = value(model.model.codebook.embedding)
    model.model.codebook.ema_count = value(model.model.codebook.ema_count)
    model.model.codebook.ema_w = value(model.model.codebook.ema_w)
end

zerograd_embedding (generic function with 1 method)

In [9]:
model_452, batch_452 = Knet.load("/home/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/nan_model_4-452.jld2", "model", "batch");

In [10]:
model_453, batch_453 = Knet.load("/home/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/nan_model_4-453.jld2", "model", "batch");

In [26]:
model = Knet.load("/home/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/nan_model_4.jld2", "prev_model");

In [11]:
check(value(model_453.model.codebook.embedding))

true

In [14]:
value(model_452.model.codebook.ema_count)[337]

1.1817311f-38

# Batch and nanification

In [15]:
nan_model = Knet.load("/home/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/nan_model_4.jld2", "model");

In [25]:
value(nan_model.model.codebook.ema_count)[337]

40.95996f0

In [16]:
check(value(nan_model.model.codebook.embedding))

false

In [14]:
nan_batch = Knet.load("/home/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/nan_model_3.jld2", "batch");

In [15]:
joined_inputs, targets, mask, terminals = atype(nan_batch[1]), atype(nan_batch[2]), atype(nan_batch[3]), atype(nan_batch[4]);

In [16]:
nan_model(joined_inputs, targets, mask, terminals)

(K32(17,24,512)[0.057556853⋯], 0.12717062f0, 0.0f0, 0.09976024f0)

In [12]:
0.12717062f0 + 0.09976024f0

0.22693086f0

# Trainer configs

In [8]:
trainer_config = Dict(
    "batch_size" => 512,
    "learning_rate" => 0.0002,
    "betas" => (0.9, 0.95),
    "weight_decay" => 0.1,
    "grad_norm_clip" => 1.0,
    "lr_decay" => false,
)

Dict{String, Any} with 6 entries:
  "betas"          => (0.9, 0.95)
  "batch_size"     => 512
  "learning_rate"  => 0.0002
  "grad_norm_clip" => 1.0
  "lr_decay"       => false
  "weight_decay"   => 0.1

In [9]:
opt_decay = AdamW(lr=trainer_config["learning_rate"], beta1=trainer_config["betas"][1], beta2=trainer_config["betas"][2], gclip=trainer_config["grad_norm_clip"])
opt_no_decay = AdamW(lr=trainer_config["learning_rate"], beta1=trainer_config["betas"][1], beta2=trainer_config["betas"][2], gclip=trainer_config["grad_norm_clip"])

for p in paramlist_decay(model)
    p.opt = clone(opt_decay)
end
for p in paramlist_no_decay(model)
    p.opt = clone(opt_no_decay)
end

# batch

In [15]:
batch = Knet.load("/home/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/nan_model_4-452.jld2", "batch");

In [16]:
joined_inputs, targets, mask, terminals = atype(batch[1]), atype(batch[2]), atype(batch[3]), atype(batch[4]);

In [20]:
loss = @diff losssum(model(joined_inputs, targets, mask, terminals))

T(NaN)

In [16]:
for p in paramlist(model)
    update!(p, grad(loss, p))
end

-0.000287451   -0.00141486    -0.00448274  …   -0.000528297   -7.38718f-5
   0.00109328    -0.00240121    -0.0140845       -0.00250097    -0.000146906
   0.000593902    0.000239694    0.00147629       0.000642225   -0.000190077
  -0.000477957   -0.00100225    -0.00400919      -0.000567076   -0.000132916
   8.31211f-5    -0.00241098    -0.0124349       -0.00294992    -0.000548493
  -0.000273197    0.000467646    0.0028253   …    0.00043963    -0.000256153
   0.000126335 

In [11]:
grad(loss, model.model.predict.w)

17×512 Knet.KnetArrays.KnetMatrix{Float32}:
 -0.000287451  -0.00141486   -0.00448274   …  -0.000528297  -7.38718f-5
  0.00109328   -0.00240121   -0.0140845       -0.00250097   -0.000146906
  0.000593902   0.000239694   0.00147629       0.000642225  -0.000190077
 -0.000477957  -0.00100225   -0.00400919      -0.000567076  -0.000132916
  8.31211f-5   -0.00241098   -0.0124349       -0.00294992   -0.000548493
 -0.000273197   0.000467646   0.0028253    …   0.00043963   -0.000256153
  0.000126335  -0.00122064   -0.00625383      -0.000363271   0.000123527
  9.15883f-5    0.000568493  -0.00690085       0.000216035   0.00103184
  0.000821024   0.00183303    0.00335294       0.00105737    0.000756728
  2.69856f-5    0.001194      0.00487685       0.00136439    0.000368385
  0.000819271   0.00285926    0.0105155    …   0.00318067    0.000397132
  0.000780148   0.0238746     0.096714         0.016012     -0.00271618
 -0.00267367    0.00777439    0.0239344        0.00506062    0.00494828
 -0.0034501

In [20]:
Knet.save("/home/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/nan_model_updated.jld2", "model", model)

# gradient check

In [12]:
probs, labels = Knet.load("/home/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/bce_inputs.jld2", "pred_terminals", "terminals")

(R(KnetArray{Float32, 3}(1,24,512)), K32(1,24,512)[0.0⋯])

In [21]:
prob_param = Param(value(probs))

P(KnetArray{Float32, 3}(1,24,512))

In [60]:
logits = Param((log.(prob_param ./ (1 .- prob_param .+ 1e-8))))

P(KnetArray{Float32, 3}(1,24,512))

In [ ]:
binary_cross_entropy(prob_param, labels)

0.0026641327f0

In [66]:
labels_cpu = Array{Int32}(reshape(labels, :))

12288-element Vector{Int32}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 1
 1

In [67]:
bce(reshape(logits, :), labels_cpu)

0.002664252f0

In [26]:
loprob_param[1,7,114]

1.0f0

In [29]:
log_pos = clip(log.(prob_param), -100.0f0, Float32(Inf))[1,7,114]

0.0f0

In [31]:
log_neg = clip(log.(1 .- prob_param), -100.0f0, Float32(Inf))[1,7,114]

-100.0f0

In [61]:
cross_entropy_loss = @diff binary_cross_entropy(prob_param, labels)

T(0.0026641327)

In [62]:
findall(x->isnan(x), cputype(grad(cross_entropy_loss, prob_param)))

1-element Vector{CartesianIndex{3}}:
 CartesianIndex(1, 7, 114)

In [68]:
bce_loss = @diff bce(reshape(logits, :), labels_cpu)

T(0.002664252)

# step through third

In [42]:
new_batch = Knet.load("/home/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/nan_model_3.jld2", "batch");
joined_inputs, targets, mask, terminals = atype(new_batch[1]), atype(new_batch[2]), atype(new_batch[3]), atype(new_batch[4]);

In [43]:
model = Knet.load("/home/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/nan_model_updated.jld2", "model");

In [44]:
v = model;

In [45]:
joined_dimension, t, b = size(joined_inputs)
padded = repeat_broadcast(v.padding_vector, 1, t, b)

if !(terminals === nothing)
    terminal_mask = repeat_broadcast(deepcopy(1 .- terminals), size(joined_inputs, 1), 1, 1)
    joined_inputs = joined_inputs .* terminal_mask .+ padded .* (1 .- terminal_mask)
end
state = joined_inputs[1:v.observation_dim, 1, :]

11×512 Knet.KnetArrays.KnetMatrix{Float32}:
  1.669      0.120672    1.95312    …   0.0218356   0.520214  -0.384414
 -1.70727    0.641554   -1.43685       -1.10089     1.42574    1.08159
 -0.637838   1.28378    -0.175449       0.349101    0.27128    0.52238
  0.550012   0.494199    0.494093       0.463235    0.684658   0.291366
 -1.45945   -0.17242     1.3172         0.962533    1.34699    0.405659
  0.983976  -1.3038      0.617697   …  -0.415012   -0.682918  -0.846424
  1.07545    0.171964   -0.335095      -0.0970336  -0.62242    0.0143702
 -0.839263   0.0313428  -0.588259      -0.929411   -2.49222   -0.455876
 -0.395215   0.35426     0.260614      -0.965969   -1.88022   -0.163415
  0.704481   0.0849387   0.0846567      0.146934   -0.275631  -0.226125
  0.445358   0.0386078  -0.20211    …   0.164299   -0.215335   0.289442

In [46]:
joined_inputs = cat(joined_inputs, terminals, dims=1);
state = state;

In [54]:
value(v.embed.w)

512×17 Knet.KnetArrays.KnetMatrix{Float32}:
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN 

In [47]:
v = model.model;

In [50]:
token_embeddings = v.embed(joined_inputs)

512×24×512 KnetArray{Float32, 3}:
[:, :, 1] =
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  Na

# step through second

In [24]:
v = model;

In [25]:
pred_trajectory = reshape(reconstructed[1:end-1, :, :], (joined_dimension, t, b))

LoadError: UndefVarError: reconstructed not defined

In [ ]:
pred_terminals = reshape(reconstructed[end, :, :], 1,size(reconstructed)[2:end]...)

In [ ]:
weights = cat(
            atype(ones(2)) .* v.position_weight,
            atype(ones(v.observation_dim - 2)),
            atype(ones(v.action_dim)) .* v.action_weight,
            atype(ones(1)) .* v.reward_weight,
            atype(ones(1)) .* v.value_weight,
            dims=1
        )

In [ ]:
mse = mse_loss(pred_trajectory, joined_inputs, reduction="none") .* reshape(weights, (:, 1, 1))

In [ ]:
function binary_cross_entropy(probs,labels; reduction="mean")
    log_pos = atype(clamp.(cputype(log.(probs)), -100.0f0, Inf))
    log_neg = atype(clamp.(cputype(log.(1 .- probs)), -100.0f0, Inf))
    loss = -(labels .* log_pos .+ (1 .- labels) .* log_neg)
    if reduction == "mean"
        mean(loss)
    elseif reduction == "sum"
        sum(loss)
    elseif reduction == "none"
        return loss
    end
end

In [ ]:
check(mse)

In [ ]:
cross_entropy = binary_cross_entropy(pred_terminals, atype(clamp.(convert.(Float32, cputype(terminals)),0.0f0, 1.0f0)))

In [ ]:
labels=atype(clamp.(convert.(Float32, cputype(terminals)),0.0f0, 1.0f0));

In [ ]:
labels .* log.(pred_terminals)

In [ ]:
x = (1 .- labels) .* log.(1 .- pred_terminals);

In [ ]:
pred_terminals[1,7,114]

In [ ]:
?clamp

In [ ]:
log_clamp(x) = log(x) < -100.0f0 ? -100.0f0 : log(x)

In [ ]:
clamp.(pred_terminals, -100.0f0, Inf)

In [ ]:
log_clamp.(1 .- pred_terminals)

In [ ]:
findall(x->isnan(x), cputype(x))

# step through

In [18]:
v = model_452;

In [19]:
joined_dimension, t, b = size(joined_inputs)
padded = repeat_broadcast(v.padding_vector, 1, t, b)

if !(terminals === nothing)
    terminal_mask = repeat_broadcast(deepcopy(1 .- terminals), size(joined_inputs, 1), 1, 1)
    joined_inputs = joined_inputs .* terminal_mask .+ padded .* (1 .- terminal_mask)
end
state = joined_inputs[1:v.observation_dim, 1, :]

11×512 Knet.KnetArrays.KnetMatrix{Float32}:
 -0.572032    -0.902455   -1.53252    …  -0.191195    0.106453  -0.160685
  0.0453874   -0.199692    0.352712      -0.0134167   1.51915   -0.726735
 -0.00753865  -0.637729   -1.16322        1.07508     1.17135    1.27078
  0.491716     0.421634    0.507299      -0.378327    0.579733  -0.371834
  0.601227     0.660066    1.25671        0.200985    0.829313   0.00663482
 -0.464948    -0.420828   -0.547654   …  -0.921333   -0.328902  -1.02853
 -0.297017    -0.0895763  -2.37184        0.0977055   1.00222    0.258226
 -1.48042     -0.0232123   0.314432      -0.411779    0.374198   0.807069
 -1.32235     -0.203968   -0.633031       0.0666929  -0.589391   0.00171385
  0.0763113   -0.275718    0.0320827     -0.451443   -0.198523   1.12484
  0.237462     0.168298    0.0380943  …   0.302337    1.99595    0.0945644

In [20]:
joined_inputs = cat(joined_inputs, terminals, dims=1);
state = state;

In [21]:
check(state)

false

In [23]:
v = model_452.model;

In [ ]:
token_embeddings = v.embed(joined_inputs)
position_embeddings = v.pos_emb[:, 1:t, :] 
x_emb = v.drop(token_embeddings .+ position_embeddings)

In [ ]:
check(x_emb)

In [ ]:
x_enc = v.encoder(x_emb)

## Attention

In [ ]:
c=v.encoder.layers[1].attn;
x=ln1_out;

In [ ]:
C, T, B = size(ln1_out)

In [ ]:
k = permutedims(reshape(c.key(x), (C ÷ c.n_head, c.n_head, T, B)), (1, 3, 2, 4));

In [ ]:
q = permutedims(reshape(c.query(x), (C ÷ c.n_head, c.n_head, T, B)), (1, 3, 2, 4));

In [ ]:
v = permutedims(reshape(c.value(x), (C ÷ c.n_head, c.n_head, T, B)), (1, 3, 2, 4));

In [ ]:
att = bmm(permutedims(k, (2,1,3,4)), q) .* Float32(1 / sqrt(size(k, 1)));

In [ ]:
att_masked = att .+ repeat_broadcast(atype((c.mask[1:T,1:T] .== 0) * Float32(-Inf)), 1,1,size(att)[end-1:end]...);

In [ ]:
function softmax(w; dims)
    w = w .- maximum(w, dims=dims)
    probs = exp.(w)
    return probs ./ sum(probs, dims=dims)
end

In [ ]:
att_softmax = softmax(att_masked, dims=1)

In [ ]:
att_masked[:,8,1,1]

In [ ]:
att_softmax[:,8,1,1]

## Rest

In [ ]:
x_pool = permutedims(v.latent_pooling(permutedims(x_enc, (2, 1, 3))), (2,1,3))

In [ ]:
x = v.cast_embed(x_pool)

In [ ]:
check(x)

In [ ]:
x

In [24]:
trajectory_feature = encode(v, joined_inputs)

512×8×512 KnetArray{Float32, 3}:
[:, :, 1] =
 -0.409032   -0.0607605  -0.298661   …   0.181049    0.14072     0.0641807
 -0.414837   -0.445586   -0.473146       0.141847   -0.101571    0.108923
  0.192495   -0.629151   -1.03216       -1.81281     1.45331    -1.73991
  0.976192    0.852615    0.285627       1.04563    -0.801759    0.671871
 -0.0388344   0.938482   -1.13685        0.521679    0.400322   -0.36328
 -0.488634   -1.24946     0.0560212  …   0.477317   -0.0218062   0.646908
  0.848961    0.447386   -0.370324      -0.74715     0.372246   -0.374788
  0.879239    0.961831    0.430717      -0.69121     0.169273   -0.519052
 -0.695879   -0.210912    0.837065       0.322585   -0.671505    0.501643
  0.172292    0.373232    0.296625       1.06767    -0.936002    0.759768
 -0.181216   -0.0810385   0.952475   …   0.839462   -0.935304    0.696749
 -0.0411872  -0.365618    0.348935       0.303721   -0.967538    0.585275
 -0.69988    -0.37797     0.370855      -0.262099    0.146547    0.4

In [25]:
check(trajectory_feature)

false

# Straight through step through

In [90]:
latents_st, latents = straight_through(v.codebook, trajectory_feature)

(K32(512,8,512)[NaN⋯], K32(512,8,512)[0.20129874⋯])

In [26]:
v = v.codebook;

In [27]:
z_e_x = trajectory_feature;

In [28]:
D, K = size(v.embedding)

(512, 512)

In [32]:
z_q_x, indices = vq_st(z_e_x, v.embedding)

(K32(512,8,512)[-0.8334601⋯], [369, 487, 149, 62, 342, 39, 218, 105, 394, 99  …  402, 494, 340, 59, 298, 262, 503, 381, 2, 389])

In [35]:
encodings = one_hot(Float32, indices, K)

512×4096 Knet.KnetArrays.KnetMatrix{Float32}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.

In [39]:
v.ema_count = v.decay .* v.ema_count + (1 - v.decay) .* sum(encodings, dims=2)[:, 1]

512-element Knet.KnetArrays.KnetVector{Float32}:
  1.3448786
 12.607567
  7.4335527
  5.5295704f-34
 10.61032
  3.277021
  1.5632615
  5.944417
 16.347204
 13.033485
 10.617094
 24.896467
  7.0121994
  ⋮
  7.9339375
 16.102657
  8.55746
  1.0580114
  6.8372784
  6.1116986
  6.3389735
  7.461581
  3.252566f-33
  6.913274
 15.359002
  4.0352488

In [40]:
v.ema_count[337]

0.0f0

In [66]:
dw = reshape(z_e_x, (D, :)) * transpose(encodings)

512×512 Knet.KnetArrays.KnetMatrix{Float32}:
 -0.798747    15.1661      18.0581    …  0.0  -2.5575     0.0   0.438663
  0.778253    -5.13471      1.38842      0.0  -0.076626   0.0  -0.866793
  1.04684    -12.1316       4.99511      0.0   0.625841   0.0  -0.343532
 -0.189358    23.5642      11.2769       0.0   8.80539    0.0  -0.155933
 -2.54666     -3.15862     -3.65597      0.0   0.925781   0.0  -1.50489
 -0.636052    10.8191      16.386     …  0.0  -2.56522    0.0  -2.63706
 -0.749141    -6.34624     -5.47774      0.0  -9.5828     0.0  -0.300571
  0.868636    -3.38916      1.28582      0.0   3.43453    0.0   2.81703
  0.257742     0.575942    -1.37473      0.0   0.345258   0.0   1.06822
  0.993109     5.5062       2.4387       0.0   2.78647    0.0   0.323353
  0.10774      5.64611     15.9018    …  0.0  -4.20387    0.0  -0.895342
  0.660078     7.25682     -5.01947      0.0   1.32413    0.0  -2.10008
 -0.536643     0.018028    -3.50547      0.0   2.08433    0.0  -0.435229
  ⋮        

In [67]:
check(dw)

false

In [68]:
v.ema_w = v.decay .* v.ema_w + (1 - v.decay) .* dw

512×512 Knet.KnetArrays.KnetMatrix{Float32}:
 -1.47407     15.8256     10.1086     …  -3.1881      0.390634   0.24742
 -0.11819     -4.62023    -0.0614944      1.08722    -5.42988   -0.575093
  1.31979    -15.007       2.61534       -0.205996    4.6602    -0.233619
  0.705494    25.0576      7.80568        7.33916    22.8024    -0.385443
 -2.28658     -2.33539    -2.83756       -1.13106     1.16046   -2.7644
 -0.777735    12.5495     10.4369     …  -1.55828   -20.6816    -3.76899
 -0.713945    -6.52543    -2.68049       -9.08471   -10.013     -1.22054
  0.782982    -3.06619     0.0189295      2.17009     4.393      3.00436
 -0.165137     0.486736   -1.33807        0.733834    6.42684    2.73043
  0.854485     6.49578     0.927457       3.06678    22.9498     0.645002
 -0.103137     5.3422      8.82978    …  -2.55801    12.6048    -0.399583
  0.256878     7.53802    -2.191          1.59068     2.75494   -2.27956
  0.0607766   -0.0461263  -0.803024       1.3232    -24.3765    -1.01542
  

In [69]:
check(v.ema_w)

false

In [72]:
v.embedding = v.ema_w ./ reshape(v.ema_count, (1, :))

512×512 Knet.KnetArrays.KnetMatrix{Float32}:
 -1.12652     1.27043      1.35602     …  -0.465042    0.0259499   0.0623956
 -0.0903241  -0.370898    -0.00824916      0.15859    -0.360709   -0.14503
  1.00862    -1.20471      0.350834       -0.0300483   0.309579   -0.0589151
  0.539157    2.01155      1.04709         1.07055     1.51477    -0.0972028
 -1.74746    -0.187478    -0.380643       -0.164986    0.0770895  -0.69714
 -0.594365    1.00744      1.40006     …  -0.227303   -1.37388    -0.950483
 -0.545615   -0.523842    -0.359573       -1.32517    -0.665167   -0.307802
  0.598375   -0.246144     0.00253929      0.316548    0.291828    0.757654
 -0.126202    0.0390737   -0.179495        0.107043    0.426937    0.688574
  0.65302     0.521461     0.124414        0.447346    1.52456     0.16266
 -0.0788196   0.428855     1.18447     …  -0.373132    0.837339   -0.100769
  0.196313    0.605129    -0.293911        0.23203     0.183012   -0.57487
  0.046447   -0.00370288  -0.107721        0

In [73]:
check(v.embedding)

false

In [74]:
z_q_x_bar_flatten = v.embedding[:, indices]
z_q_x_bar = reshape(z_q_x_bar_flatten, size(z_e_x))

512×8×512 KnetArray{Float32, 3}:
[:, :, 1] =
 -0.457951    0.239132   -0.204369   …  -0.30512    -0.123156    0.44504
 -0.352019   -0.68077    -0.390198      -0.216223   -0.766081   -0.627818
 -1.20884    -1.42021    -1.08851       -0.38463    -0.333794    0.744214
  0.999057    0.452675    0.91135        0.882649    1.06058     0.886452
  0.197041   -0.506171   -0.444217      -0.244133   -0.741016   -0.420892
 -0.265307   -0.363584   -0.552367   …  -1.22309     0.731109    0.785565
  0.393205   -0.38334    -0.267159      -0.0624272  -0.610513   -0.26626
  0.390033    0.0786816   0.5734         0.872762    0.113167   -0.246663
  0.0628653   1.26152     0.517243       0.327403   -0.41426    -0.403565
  0.708179    0.0349848   0.392282       0.398417   -0.138435   -0.627524
  0.670886    0.287661    0.4408     …   0.323531   -0.401626   -0.136025
  0.451922    0.355511    0.371607       0.058394    0.297751    0.00469918
  0.522959    0.33462     0.144795      -0.0823849   0.424721    0.